# Stacking submissions with different preprocessing
Final submission. Stacking for estimators with their preferred parametrization.

In [6]:
%matplotlib inline
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# set seeds
np.random.seed(1)
random.seed(1)

# path data
path_data = 'data/'

# Clean data

## Create and drop features, manage categorical

In [7]:
# Function to prepare data

from sklearn.base import TransformerMixin

def create_new_features(X):
    # transform utc offset:
    X['Sin_UTC'] = np.sin((11 * 3600 + X['UTC Offset']) / (24 * 3600) * 2 * np.pi)
    X['Cos_UTC'] = np.cos((11 * 3600 + X['UTC Offset']) / (24 * 3600) * 2 * np.pi)
    
    # time since creation in days
    duration = (pd.to_datetime('today') - 
                pd.to_datetime(X['Profile Creation Timestamp']).dt.tz_localize(None))
    X['Duration'] = duration.apply(lambda x: x.days)
    
    # convert personal url into True/False (NaNs or unique)
    X['Has Personal URL'] = X['Personal URL'].notna()
    
    # log(x + 1) of numerical features
    for feature in ['Num of Followers', 'Num of People Following',
                    'Num of Status Updates', 'Num of Direct Messages',
                    'Avg Daily Profile Visit Duration in seconds',
                    'Avg Daily Profile Clicks']:
        X[f'log {feature}'] = np.log1p(X[feature])
    return X
    
def clean_features(X):
    # merge categories names with and without cap letter
    X['Location Public Visibility'] = X['Location Public Visibility'].str.lower()
    X.loc[X['Profile Category'] == " ", 'Profile Category'] = 'unknown'
    return X

def drop_features(X, l_features):
    return X.drop(columns=l_features)
    
class RemoveCategories(TransformerMixin):
    def __init__(self, min_obs=10, l_cols=[]):
        self.min_obs = min_obs
        self.l_cats = {}
        self.l_cols = l_cols

    def fit(self, X):
        # assumes all columns of df_cat are strings
        for col in self.l_cols:
            val_counts = X[col].fillna('missing').value_counts()
            self.l_cats[col] = val_counts[val_counts >= self.min_obs].index.tolist()
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        for col in self.l_cols:
            X.loc[~ X[col].isin(self.l_cats[col]), col] = 'other'        
        return X

In [8]:
# data cleaning and feature engineering

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

l_cat_small = ['Profile Text Color', 'Profile Page Color',
               'Profile Theme Color', 'User Language', 'Location']
l_drop = ['Id', 'User Name', 'Profile Image', 'Personal URL',
          'Profile Creation Timestamp', 'Location', 'UTC Offset']
l_drop_logs = ['Num of Followers', 'Num of People Following',
               'Num of Status Updates', 'Num of Direct Messages',
               'Avg Daily Profile Visit Duration in seconds', 
               'Avg Daily Profile Clicks']

features_to_drop =dict(l_features = l_drop + l_drop_logs)

pipe = Pipeline([('create new features',
                  FunctionTransformer(create_new_features)),
                 ('clean data', FunctionTransformer(clean_features)),
                 ('remove categories', RemoveCategories(min_obs=2, 
                                                        l_cols=l_cat_small)),
                 ('drop variables', FunctionTransformer(drop_features,
                                    kw_args=features_to_drop))
                ])

## Imputation and encoding/scaling

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

# numerical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
    ])

numeric_transformer_poly = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2))
    ])

# categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [10]:
# train data
df = pd.read_csv(path_data +'train.csv')
y = np.log1p(df['Num of Profile Likes'].values)
X = df.drop(columns='Num of Profile Likes')

X = pipe.fit_transform(X)

preprocessor_poly = ColumnTransformer(
    transformers=[('num', numeric_transformer_poly,
                   X.select_dtypes(exclude=['object', 'bool']).columns),
                  ('cat', categorical_transformer,
                   X.select_dtypes(include=['object', 'bool']).columns)])

preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer,
                   X.select_dtypes(exclude=['object', 'bool']).columns),
                  ('cat', categorical_transformer,
                   X.select_dtypes(include=['object', 'bool']).columns)])

# Stacking

In [12]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

pipe_ridge = Pipeline([('preprocess_poly', preprocessor_poly), 
                       ('ridge', Ridge(alpha=48, normalize=False))])
pipe_rf = Pipeline([('preprocess', preprocessor), 
                    ('rf', RandomForestRegressor(n_estimators=700, min_samples_split=3,
                                                 min_samples_leaf=3, max_depth=18))])
pipe_gb = Pipeline([('preprocess', preprocessor), 
                    ('gb', GradientBoostingRegressor(n_iter_no_change=6, n_estimators=10000,
                                                     min_samples_split=21, min_samples_leaf=9,
                                                     max_depth=5, learning_rate=0.06))])
pipe_svr = Pipeline([('preprocess', preprocessor), 
                    ('svr', SVR(kernel='rbf', C=1.37))])

estimators = [
    ('ridge', pipe_ridge),
    ('rf', pipe_rf),
    ('gb', pipe_gb),
    ('svr', pipe_svr)
]

In [14]:
stack = StackingRegressor(final_estimator=RidgeCV(alphas=np.logspace(-2, 2, 5),
                                                  normalize=True, store_cv_values=True),
                          estimators=estimators, n_jobs=200,
                          passthrough=False, verbose=10)
stack.fit(X, y)

coeffs = [np.round(val, 3) for val in stack.final_estimator_.coef_]
RMSE = np.round(np.sqrt(np.mean(stack.final_estimator_.cv_values_, 0)), 4)

print(f'intercept and coefficients: {stack.final_estimator_.intercept_:.3f}, {coeffs}')
print(f'alphas: {stack.final_estimator_.alphas} and scores: {RMSE}')

intercept and coefficients: -0.337, [0.241, 0.206, 0.339, 0.259]
alphas: [1.e-02 1.e-01 1.e+00 1.e+01 1.e+02] and scores: [1.7    1.7005 1.7447 2.1916 2.5104]


In [11]:
def create_submission(estimator, number):
    X_test = pd.read_csv(path_data + 'test.csv')
    Id = X_test['Id']
    X_test = pipe.transform(X_test)
    X_test = preprocessor.transform(X_test)
    y_pred = (estimator.predict(X_test))
    y_pred_test = pd.Series(np.expm1(y_pred))
    submission = pd.DataFrame({'Id': Id, 'Predicted': y_pred_test})
    path = f'submissions/submission{number}.csv'
    while os.path.exists(path):
        number += 1
        path = f'submissions/submission{number}.csv'
    submission.to_csv(path, index=False)
    print(f'submission #{number} created')

In [ ]:
create_submission(stack, 25)